In [1]:
import os
from twython import Twython
from twython import TwythonStreamer

In [ ]:
CONSUMER_KEY = 'YOUR CONSUMER KEY'
CONSUMER_SECRET = 'YOUR CONSUMER SECRET'
ACCESS_TOKEN = 'YOUR ACCESS TOKEN'
ACCESS_SECRET = 'YOUR ACCESS SECRET'

In [ ]:
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET,
                  ACCESS_TOKEN, ACCESS_SECRET)

In [ ]:
current_dir = os.getcwd()
os.chdir('..')
base_path = os.getcwd()
data_folder = (os.path.join(base_path,'data'))
output_folder = (os.path.join(base_path,'outputs'))

In [ ]:
os.chdir(data_folder)

In [ ]:
nps = pd.read_csv('np_info_full_park.csv')


handles = nps['Handle'].values
parks = nps['Park'].values

park_df = []
park_list = nps['Handle'].values
for item in range(len(park_list)):
    if nps['Handle'].isnull()[item]:
        park_frame = pd.DataFrame({'Park':parks[item],'Handle':handles[item],
                                   'Users':0},
                                  index=[item])
        park_df.append(park_frame)
        
    else:
        park_frame = pd.DataFrame({'Park':parks[item],'Handle':handles[item],
                                   'Users':twitter.lookup_user(screen_name=handles[item])[0]['followers_count']},
                                  index=[item])
        park_df.append(park_frame)
    
    
combined_frame = pd.concat(park_df)

#merge with counts
nps_with_users = pd.merge(left=nps,right=combined_frame,how='left',on = 'Park')
nps_with_users.drop(['Handle_y'],axis=1,inplace=True)
nps_with_users.rename(columns={'Handle_x':'Handle'},inplace=True)
nps_with_users['Handle'].fillna('No Handle',inplace=True)